In [1]:
"""username = "nova"
pathtorepo = "/home/nova/work/repo_ds/dynamical-systems"
using Pkg
Pkg.activate(pathtorepo * "/env/integrate/")

using StaticArrays, DifferentialEquations, DynamicalSystems, IntervalRootFinding 
using CairoMakie, GLMakie

include("/home/nova/work/repo_ds/dynamical-systems/system.jl")
include("/home/nova/work/repo_ds/dynamical-systems/visual.jl")"""

"username = \"nova\"\npathtorepo = \"/home/nova/work/repo_ds/dynamical-systems\"\nusing Pkg\nPkg.activate(pathtorepo * \"/env/integrate/\")\n\nusing StaticArrays, DifferentialEquations, DynamicalSystems, IntervalRootFinding \nusing CairoMakie, GLMakie\n\ninclude(\"/home/nova/work/repo_ds/dynamical-systems/system.jl\")\ninclude(\"/home/nova/work/repo_ds/dynamical-systems/visual.jl\")"

In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems

include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\system.jl");
#include("C:\\Users\\Alex\\Desktop\\dynamical-systems\\visual.jl");

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

TM6_glial_ECM_help (generic function with 1 method)

In [2]:
tLSE = 3000.0; tt = 0.0; tstep = 0.001;
tol = 1e-15;
integ_set = (alg = Vern9(), adaptive = true, abstol = tol, reltol = tol);

In [3]:
param = TM6_glial_ECM_get_params();
dict, indexparams = TM6_glial_ECM_help(param);
println(indexparams);


    τ - 1, τD - 2, τF - 3, τy - 4, α - 5, αE - 6, αecm - 7, αp - 8,
    J - 9, U0 - 10, I0 - 11, ΔU0 - 12, β - 13, βecm  -14, βp - 15,
    γp - 16, ecm0 - 17, ecm1 - 18, kecm - 19, θecm - 20, p0 - 21, p1 - 22, θp - 23, kp - 24, ythr - 25, xthr - 26


In [13]:
u0 = SA[0.9445509341100914, 0.74116702856987, 0.7361196042973006, 0.0646914552140727, 0.15145764079879162, 0.0009327645775731449]; 
#zeros(6); #[0.9445509341100914, 0.74116702856987, 0.7361196042973006, 0.0646914552140727, 0.15145764079879162, 0.0009327645775731449]
param[11] =  -1.741;
param[6] = 4; #0.067;

In [14]:
ds= CoupledODEs(TM6_glial_ECM, u0, param, diffeq = integ_set);

In [15]:
function lyapunovspectrum_adapt(ds, time_lse)
    coeff_adapt_time = 1;
    flag = true;
    tands = TangentDynamicalSystem(ds; J = TM6_glial_ECM_jac);
    Λs = lyapunovspectrum(tands, time_lse);
    while flag == true
        checkzero = find_zero(Λs);
        if checkzero == -1
            coeff_adapt_time*=2;
            Λs = lyapunovspectrum(tands, time_lse * coeff_adapt_time);
        else
            flag = false;
        end
    end
    return Λs, coeff_adapt_time;
end

function find_zero(Λs; tol = 1e-3)
    vec_bool = Vector{Bool}(undef, 6);
    for (index, lp) in enumerate(Λs)
        checknull=  isapprox(lp, 0.0; atol = tol);
        vec_bool[index] = checknull;
    end
    vec_of_true = findall(x->x==true, vec_bool);
    if length(vec_of_true) >=1
        return 0;
    else
        return -1;
    end;
end

find_zero (generic function with 1 method)

In [16]:
lse, coeff = lyapunovspectrum_adapt(ds, 1000)

([-0.0005822638980722053, -0.01148801516405035, -0.27320510759655137, -0.3939160863979931, -0.7002932560249261, -3.562668692321662], 128)